# Calculate the external dependency factor for issues <span style="color:red">after refinement</span> for the given report variables

In [1]:
# Change directory to workspace root so that relative path loads work correctly.
from IPython.display import Markdown as md
import os
import datetime
try:
    os.chdir(os.path.join(os.getcwd(), '../'))
    print(os.getcwd())
except:
    pass

from ia.common.jira.connection import connect
import ia.dependency.conf.components as components
import ia.dependency.algo as dependency
import ia.dependency.metrics_store as metrics
import ia.common.viz.conf.dashboard as confboard


report_start_time = datetime.datetime.now()
report_timestamp = f'`Report executed at {report_start_time:%Y-%m-%d %H:%M}`'
md(report_timestamp)

/Users/lswolkien/Devel/EngineeringMetrics


`Report executed at 2020-07-03 15:04`

# Report variables

In [2]:
conf_space_key = "~lswolkien"
conf_parent_page = "Reports"
conf_summary_page = "Independence summary"

projects = {
    'DANCOE': "DANCOE - Dependencies after refinement",
    'DANMR': "DANMR - Dependencies after refinement",
    'DANSDP': "DANSDP - Dependencies after refinement",
    'DANTF': "DANTF - Dependencies after refinement",
    'DANMIG': "DANMIG - Dependencies after refinement",
#     'DANDI': "DANDI - Dependencies after refinement",
    'DANMRGCORE': "DANMRGCORE - Dependencies after refinement",
    "DANUNCO": "DANUNCO - Dependencies after refinement",
    "DANSRE": "DANSRE - Dependencies after refinement",
}
report_title='External dependency factor for issues after refinement' 
report_description="The number of issues with external dependencies to the total number of issues not Done yet,\
but after refinement (in the backlog or in the sprint)"


# Access variables

In [3]:
jira_url = os.environ['JIRA_URL']
jira_username = os.environ['JIRA_USERNAME']
jira_password = os.environ['JIRA_PASSWORD']

conf_url = os.environ['CONFLUENCE_URL']
conf_username = os.environ['CONFLUENCE_USERNAME']
conf_password = os.environ['CONFLUENCE_PASSWORD']

## Update metrics

In [4]:
dashboard = confboard.Dashboard(conf_url, conf_username, conf_password)
jira =  connect(jira_url, basic_auth=(jira_username, jira_password))
metrics_history = {}
for jira_project_id in projects.keys():
    jql = f'project = {jira_project_id} AND issuetype not in ("Test", "Sub-Task", "Release", "Risk", "Incident") and status not in ("Done", "In Analysis", "Closed", "Open")'
    p, all_issues, all_with_dep = dependency.dependency_factor(jira, jql)
    independency_factor = 100-p

    metrics.save('after refinement', jira_project_id, independency_factor, all_issues, all_with_dep)
    metrics_history[jira_project_id] = metrics.read_independence_stats('after refinement', jira_project_id)

    report_head = confboard.Component(confboard.report_head, [report_title, report_description])
    report_dependency = confboard.Component(
        components.dependency_report, 
        [independency_factor, all_issues, all_with_dep, metrics_history[jira_project_id]]
    )

    conf_page_title = projects[jira_project_id]
    dashboard.publish([report_head, report_dependency], conf_page_title, conf_space_key, conf_parent_page)

report_head = confboard.Component(confboard.report_head, ["Independence after refinement", None])
report_summary = confboard.Component(components.dependency_summary, [jira, metrics_history, projects.keys()])



legend_1 = "searches for 'external' dependencies that can be direct or indirect (internal dependencies are ignored)"
legend_2 = "issuetype not in ('Test', 'Sub-Task', 'Release', 'Risk', 'Incident')"
legend_3 = "status not in ('Done', 'In Analysis', 'Open')'. We only consider work items after refinement"
legend_4 = "link type in ('Is blocked by', or 'Depends on')."
legend_5 = "related dependencies are filtered if their status is 'Done'."
legend_6 = "plus Jira items in the Blocked state that don't have links. This is taken into account because there are teams that do not use Jira"

report_legend = confboard.Component(confboard.report_legend, [
    [legend_1, legend_2, legend_3, legend_4, legend_5, legend_6], "The dependency factor legend:"
])

dashboard.publish([report_head, report_legend, report_summary], conf_summary_page, conf_space_key, conf_parent_page)


In [5]:
report_end_time = datetime.datetime.now()
report_timestamp = f'` Report finished at {report_end_time:%Y-%m-%d %H:%M}, time elapsed: {report_end_time - report_start_time}`'
md(report_timestamp)


` Report finished at 2020-07-03 15:07, time elapsed: 0:03:01.267066`

## Tear down

In [6]:

import glob
files = glob.glob('*.png')
for filename in files:
    try:
        os.remove(filename)
    except:
        print("Error while deleting file : ", filename)